In [99]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [100]:
books["genre"].value_counts().reset_index()

,genre,count
0,"Mystery,Fiction,Mystery,Crime,Classics,Mystery...",7
1,"Sequential Art,Graphic Novels,Sequential Art,C...",6
2,"Romance,Historical Romance,Romance,Historical,...",5
3,"Sequential Art,Comics,Sequential Art,Graphic N...",5
4,"Fantasy,Fiction,Fantasy,Epic Fantasy,Fantasy,H...",4
...,...,...
22851,"Language,Writing,Nonfiction,Reference,Self Hel...",1
22852,"Education,Nonfiction,Parenting,Teaching,Childr...",1
22853,"Science Fiction,Fiction,Space,Space Opera,Myst...",1
22854,"Historical,Historical Fiction,Young Adult,Fict...",1


In [102]:
books["genre"].value_counts().reset_index().query("count > 4")

,genre,count
0,"Mystery,Fiction,Mystery,Crime,Classics,Mystery...",7
1,"Sequential Art,Graphic Novels,Sequential Art,C...",6
2,"Romance,Historical Romance,Romance,Historical,...",5
3,"Sequential Art,Comics,Sequential Art,Graphic N...",5


In [103]:
books[books["genre"]=="Art,Crafts"]

,author,desc,genre,img,isbn,link,pages,rating,reviews,title,totalratings,tagged_description


In [107]:
category_mapping = {
  "Nonfiction": "Nonfiction",
  "History": "Nonfiction",
  "Games,Chess": "Nonfiction",
  "Esoterica,Astrology": "Nonfiction",
  "History,Nonfiction": "Nonfiction",
  "Music": "Nonfiction",
  "Combat,Martial Arts": "Nonfiction",
  "Crafts,Quilting": "Nonfiction",
  "Science,Mathematics": "Nonfiction",
  "Art": "Nonfiction",
  "Poetry": "Fiction",
  "Nurses,Nursing": "Nonfiction",
  "Fiction": "Fiction",
  "Occult,Tarot": "Nonfiction",
  "Romance,Romance,African American Romance": "Fiction",
  "Childrens": "Fiction",
  "Reference": "Nonfiction",
  "Alcohol,Wine": "Nonfiction",
  "Philosophy": "Nonfiction",
  "Romance": "Fiction",
  "Literature,Marathi": "Fiction",
  "Crafts,Origami": "Nonfiction",
  "Architecture": "Nonfiction",
  "Nonfiction,History": "Nonfiction",
  "Travel": "Nonfiction",
  "Science": "Nonfiction",
  "Romance,African American Romance": "Fiction",
  "Crafts,Sewing": "Nonfiction",
  "Cultural,Africa": "Nonfiction",
  "Spirituality": "Nonfiction",
  "Crafts,Knitting,Art,Crafts,Nonfiction": "Nonfiction",
  "Social Science,Social Work": "Nonfiction",
  "Food and Drink,Cookbooks": "Nonfiction",
  "Romance,African American Romance,Romance": "Fiction",
  "Gardening,Nonfiction": "Nonfiction",
  "Couture,Fashion": "Nonfiction",
  "Childrens,Picture Books,Childrens": "Fiction",
  "Aviation": "Nonfiction",
  "Crafts,Knitting,Nonfiction,Art,Crafts": "Nonfiction",
  "Gardening": "Nonfiction",
  "Crafts,Crochet": "Nonfiction",
  "Games,Role Playing Games": "Fiction",
  "Biography": "Nonfiction",
  "Business": "Nonfiction",
  "Religion": "Nonfiction",
  "Science,Chemistry": "Nonfiction",
  "Art,Art,Drawing": "Nonfiction",
  "Labor": "Nonfiction",
  "Art,Crafts": "Nonfiction",
  "Sports,Cycling": "Nonfiction",
  "Sports,Baseball,Sports,Sports,Nonfiction": "Nonfiction",
  "Fantasy": "Fiction",
  "Mystery": "Fiction",
  "Cultural,Iran": "Nonfiction",
  "Childrens,Picture Books": "Fiction",
  "Harlequin,Harlequin Romance": "Fiction",
  "Games,Role Playing Games,Fantasy": "Fiction",
  "Westerns":"Fiction",
  "Westerns,Fiction": "Fiction",
  "Asian Literature,Turkish Literature": "Fiction",
  "Horror": "Fiction"
}


In [108]:
books["simple_categories"] = books["genre"].map(category_mapping)

In [109]:
books[~(books["simple_categories"].isna())]

,author,desc,genre,img,isbn,link,pages,rating,reviews,title,totalratings,tagged_description,simple_categories
12253,"Krishna Dharabasi,à¤•à¥ƒà¤·à¥à¤£ à¤§à¤°à¤¾à¤µ...",Radha (à¤°à¤¾à¤§à¤¾) represents an example of ...,Fiction,https://i.gr-assets.com/images/S/compressed.ph...,9993329053,https://goodreads.com/book/show/13445136-radha,262,4.17,64,à¤°à¤¾à¤§à¤¾ [Radha],736,9993329053 Radha (à¤°à¤¾à¤§à¤¾) represents an ...,Fiction


In [110]:
fine_tuning_df = books[~(books["simple_categories"].isna())]

In [111]:
fine_tuning_df = fine_tuning_df[["desc", "simple_categories"]]

In [112]:
fine_tuning_df

,desc,simple_categories
12253,Radha (à¤°à¤¾à¤§à¤¾) represents an example of ...,Fiction


In [113]:
fine_tuning_df[fine_tuning_df["simple_categories"]=="Fiction"]

,desc,simple_categories
12253,Radha (à¤°à¤¾à¤§à¤¾) represents an example of ...,Fiction


In [114]:
fine_tuning_df.to_csv("fine_tuning_df.csv", index = False)

In [115]:
from transformers.pipelines import pipeline

fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification", model="MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli",)

Device set to use cpu


In [117]:
sequence = books.loc[books["simple_categories"] == "Fiction", "desc"].reset_index(drop=True)[0]

In [88]:
pipe(sequence, fiction_categories)

{'sequence': "Injured in a cattle stampede on a drive through Kansas, Bill is healing in Redemption. Tended to by a shopkeeper's lovely daughter, Eden Monroe, there are worse places he could be. But Bill knows something's wrong. A series of unexplained killings plagues the town. It's up to Bill to figure out what's going on before his beloved Eden gets caught in the cross fire.",
 'labels': ['Fiction', 'Nonfiction'],
 'scores': [0.9987571239471436, 0.0012428824556991458]}

In [118]:
import numpy as np

max_index = np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label = pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [119]:
def generate_predictions(sequence, categories):
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [ ]:
from tqdm import tqdm

actual_cats = []
predicted_cats = []

for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Fiction", "desc"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Fiction"]

In [92]:
for i in tqdm(range(0, 300)):
    sequence = books.loc[books["simple_categories"] == "Nonfiction", "desc"].reset_index(drop=True)[i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    actual_cats += ["Nonfiction"]

100%|██████████| 300/300 [09:55<00:00,  1.98s/it]


In [93]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})

In [94]:
predictions_df

,actual_categories,predicted_categories
0,Fiction,Fiction
1,Fiction,Nonfiction
2,Fiction,Fiction
3,Fiction,Fiction
4,Fiction,Fiction
...,...,...
595,Nonfiction,Fiction
596,Nonfiction,Nonfiction
597,Nonfiction,Nonfiction
598,Nonfiction,Fiction


In [95]:
predictions_df["correct_prediction"] = (
    np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
)

In [96]:
predictions_df["correct_prediction"].sum() / len(predictions_df)

np.float64(0.8683333333333333)

In [121]:
isbns = []
predicted_cats = []

missing_cats = books.loc[books["simple_categories"].isna(), ["isbn", "desc"]].reset_index(drop=True)

In [ ]:
for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["desc"][i]
    predicted_cats += [generate_predictions(sequence, fiction_categories)]
    isbns += [missing_cats["isbn"][i]]

In [ ]:
missing_predicted_df = pd.DataFrame({"isbn": isbns, "predicted_categories": predicted_cats})

In [ ]:
missing_predicted_df

In [ ]:
books = pd.merge(books, missing_predicted_df, on="isbn", how="left")
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])
books = books.drop(columns = ["predicted_categories"])

In [ ]:
books.to_csv("books_with_categories.csv", index=False)